In [8]:
# import libraries
import datetime
from datetime import datetime
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import random
import warnings
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [9]:
df = pd.read_csv('exports/stocks.csv')

In [10]:
# Add ".KL" to each value in the 'stock_code' column
testdf = df['stock_code'].apply(lambda x: x + ".KL")
testdf

0         5250.KL
1       0800EA.KL
2         7167.KL
3         7086.KL
4         5198.KL
5        03028.KL
6         7131.KL
7         0218.KL
8         0122.KL
9         1481.KL
10        5281.KL
11        9148.KL
12        7191.KL
13        7146.KL
14        0181.KL
15        6599.KL
16        5139.KL
17        5185.KL
18        7145.KL
19        0258.KL
20        7315.KL
21        7078.KL
22        0209.KL
23        5238.KL
24        2658.KL
25        7609.KL
26        5116.KL
27        5115.KL
28        2674.KL
29        0079.KL
30        2488.KL
31        1163.KL
32       03051.KL
33        5269.KL
34        5127.KL
35        5293.KL
36        5307.KL
37        5120.KL
38       03011.KL
39        1015.KL
40        7031.KL
41        6351.KL
42        7083.KL
43        0048.KL
44        4758.KL
45        0226.KL
46        6556.KL
47        5082.KL
48        5568.KL
49        5088.KL
50        7090.KL
51        5015.KL
52        6432.KL
53        0119.KL
54        7214.KL
55        

In [11]:
# Put it into a list called stock_code
stock_code = list(testdf)
stock_code

['5250.KL',
 '0800EA.KL',
 '7167.KL',
 '7086.KL',
 '5198.KL',
 '03028.KL',
 '7131.KL',
 '0218.KL',
 '0122.KL',
 '1481.KL',
 '5281.KL',
 '9148.KL',
 '7191.KL',
 '7146.KL',
 '0181.KL',
 '6599.KL',
 '5139.KL',
 '5185.KL',
 '7145.KL',
 '0258.KL',
 '7315.KL',
 '7078.KL',
 '0209.KL',
 '5238.KL',
 '2658.KL',
 '7609.KL',
 '5116.KL',
 '5115.KL',
 '2674.KL',
 '0079.KL',
 '2488.KL',
 '1163.KL',
 '03051.KL',
 '5269.KL',
 '5127.KL',
 '5293.KL',
 '5307.KL',
 '5120.KL',
 '03011.KL',
 '1015.KL',
 '7031.KL',
 '6351.KL',
 '7083.KL',
 '0048.KL',
 '4758.KL',
 '0226.KL',
 '6556.KL',
 '5082.KL',
 '5568.KL',
 '5088.KL',
 '7090.KL',
 '5015.KL',
 '6432.KL',
 '0119.KL',
 '7214.KL',
 '7181.KL',
 '7007.KL',
 '0038.KL',
 '0068.KL',
 '7722.KL',
 '7129.KL',
 '0159.KL',
 '0105.KL',
 '4057.KL',
 '7162.KL',
 '7054.KL',
 '03032.KL',
 '6399.KL',
 '0072.KL',
 '8176.KL',
 '7048.KL',
 '5130.KL',
 '7099.KL',
 '5302.KL',
 '03037.KL',
 '8885.KL',
 '5204.KL',
 '7579.KL',
 '6888.KL',
 '5106.KL',
 '7120.KL',
 '2305.KL',
 '5021.KL

In [12]:
def convert_date_string_to_datetime(date_str):
    date_format = "%d %b %Y"
    
    try:
        date_obj = datetime.strptime(date_str, date_format)
        return date_obj
    except ValueError:
        return None


In [13]:
driver = webdriver.Chrome()
driver.implicitly_wait(20)
driver2 = webdriver.Chrome()
driver2.implicitly_wait(20)
driver3 = webdriver.Chrome()
driver3.implicitly_wait(20)

# funtion to get link
def get_link(stock_code):
    
    url='https://www.bursamarketplace.com/index.php'
    driver.get(url)
    
    # Clicking the Search Button
    s= driver.find_element(By.XPATH,'//*[@id="newnav-mobileSearch"]')
    driver.execute_script("arguments[0].click();",s)
    
    # Inputting the Stock Code in the Search Bar
    driver.find_element(By.XPATH, '//*[@id="newnav-search-input"]').send_keys(stock_code)
    
    # Getting the Link in the appeared Search Results
    lnks = driver.find_element(By.ID,'url')
    links=lnks.get_attribute('href')
    return links

In [14]:
results_df = []
new_list = []

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=["Date", "Revenue"])

# Loop through each ticker in the CSV
for ticker_symbol in stock_code:
    try:
        url_base=get_link(ticker_symbol[:-3]) # Exclude ".KL for Bursa Marketplace Search Function"
        url = url_base + '/financials_IS_Qt'  # Append the desired subpage to the URL
        url2 = url_base + '/financials_BS_Qt'
        url3 = url_base + '/financials_CF_Qt'
        driver.get(url)
        driver2.get(url2)
        driver3.get(url3)
        content = driver.page_source
        content2 = driver2.page_source
        content3 = driver3.page_source
        soup = bs(content)
        soup2 = bs(content2)
        soup3 = bs(content3)

        time.sleep(2)

        # Initialize the row dictionary to store the data
        row = {}

        # Finding the Element to format the date
        first_sibling = driver.find_element(By.XPATH, "//div[text()='MYR (MILLION)']")
        first_sibling_year = driver.find_element(By.XPATH, "//div[contains(@class, 'tb_cell tb_metr')]")

        # Find the element with the Financial details span text
        try:
            revenue_prev_element = driver.find_element(By.XPATH, "//div[span='Total Revenue']")
            operating_income_prev_element = driver.find_element(By.XPATH, "//div[span='Operating Income']")
        except Exception as e:
            print(f"Revenue or Operating Income not found, stock code {ticker_symbol} is a bank. Proceed to scrape Net Interest Income and Total Earning Assets")
            revenue_prev_element = driver2.find_element(By.XPATH, "//div[span='Other Earning Assets, Total']")
            operating_income_prev_element = driver.find_element(By.XPATH, "//div[span='Net Interest Income']")
        preferred_dividends_prev_element = driver.find_element(By.XPATH, "//div[span='Preferred Dividends']")
        net_income_prev_element = driver.find_element(By.XPATH, "//div[span='Net Income']")
        # gross_dividend_prev_element = driver.find_element(By.XPATH, "//div[span='Gross Dividends - Common Stock']")
        total_equity_prev_element = driver2.find_element(By.XPATH, "//div[span='Total Equity']")
        shares_outstanding_prev_element = driver2.find_element(By.XPATH, "//div[span='Total Common Shares Outstanding']")
        operating_cf_prev_element = driver3.find_element(By.XPATH, "//div[span='Cash from Operating Activities']")
        capital_expenditures_prev_element = driver3.find_element(By.XPATH, "//div[span='Capital Expenditures']")
        gross_dividend_prev_element = driver3.find_element(By.XPATH, "//div[span='Cash Dividends Paid - Common']")
        
        
        # Iterate through the next 4 siblings to get the latest 4 quarter dates
        quarter_dates = []

        for i in range(8):
            try:
                quarter_date = first_sibling.find_element(By.XPATH, "following-sibling::div")
                year = first_sibling_year.find_element(By.XPATH, "following-sibling::div")
            except Exception as e:
                print(f"Stock code {ticker_symbol} does not have up to 8 quarterly reports at this time, proceeding with {i} quarters of available data.")
                break  # Add this line to break out of the loop
            if not year.text.strip():
                if previous_year:
                    current_date = quarter_date.text + " " + previous_year.text
            else:
                current_date = quarter_date.text + " " + year.text
                previous_year = year  # Update the previous_year

            date_obj = convert_date_string_to_datetime(current_date)

            revenue_element = revenue_prev_element.find_element(By.XPATH, "following-sibling::div")
            operating_income_element = operating_income_prev_element.find_element(By.XPATH, "following-sibling::div")
            preferred_dividends_element = preferred_dividends_prev_element.find_element(By.XPATH, "following-sibling::div")
            net_income_element = net_income_prev_element.find_element(By.XPATH, "following-sibling::div")
            gross_dividend_element = gross_dividend_prev_element.find_element(By.XPATH, "following-sibling::div")
            total_equity_element = total_equity_prev_element.find_element(By.XPATH, "following-sibling::div")
            shares_outstanding_element = shares_outstanding_prev_element.find_element(By.XPATH, "following-sibling::div")
            operating_cf_element = operating_cf_prev_element.find_element(By.XPATH, "following-sibling::div")
            capital_expenditures_element = capital_expenditures_prev_element.find_element(By.XPATH, "following-sibling::div")
            
            row["Date"] = date_obj
            row["Revenue"] = revenue_element.text
            row["Preferred Dividends"] = preferred_dividends_element.text
            row["Operating Income"] = operating_income_element.text
            row["Net Income"] = net_income_element.text
            row["Gross Dividend"] = gross_dividend_element.text
            row["Total Equity"] = total_equity_element.text
            row["Shares Outstanding"] = shares_outstanding_element.text
            row["Operating Cash Flow"] = operating_cf_element.text
            row["Capital Expenditures"] = capital_expenditures_element.text
            row["stock_code"] = ticker_symbol[:-3]

            first_sibling_year = year
            first_sibling = quarter_date
            revenue_prev_element = revenue_element
            preferred_dividends_prev_element = preferred_dividends_element
            operating_income_prev_element = operating_income_element
            net_income_prev_element = net_income_element
            gross_dividend_prev_element = gross_dividend_element
            total_equity_prev_element = total_equity_element
            shares_outstanding_prev_element = shares_outstanding_element
            operating_cf_prev_element = operating_cf_element
            capital_expenditures_prev_element = capital_expenditures_element
            
            # Append the row to the results DataFrame
            results_df = results_df.append(row, ignore_index=True)
            
    except Exception as e:
        print(f"Error fetching data for {ticker_symbol}: {str(e)}")
        new_list.append(ticker_symbol)
  

# Assuming 'results_df' is your DataFrame
numeric_columns = results_df.columns.difference(['Date', 'stock_code'])
results_df[numeric_columns] = results_df[numeric_columns].replace({',': ''}, regex=True)

# Convert all columns to float except 'Date' and 'stock_code'
results_df = results_df.apply(lambda col: pd.to_numeric(col, errors='coerce') if col.name not in ['Date', 'stock_code'] else col)

# Fill NaN values with 0
results_df = results_df.fillna(0)
print(results_df)
    




C:\Users\Aidan\AppData\Local\Temp\ipykernel_14884\3273120891.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_14884\3273120891.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_14884\3273120891.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_14884\3273120891.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = result

Error fetching data for 0800EA.KL: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[text()='MYR (MILLION)']"}
  (Session info: chrome=119.0.6045.105); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7E45D82B2+55298]
	(No symbol) [0x00007FF7E4545E02]
	(No symbol) [0x00007FF7E44005AB]
	(No symbol) [0x00007FF7E444175C]
	(No symbol) [0x00007FF7E44418DC]
	(No symbol) [0x00007FF7E447CBC7]
	(No symbol) [0x00007FF7E44620EF]
	(No symbol) [0x00007FF7E447AAA4]
	(No symbol) [0x00007FF7E4461E83]
	(No symbol) [0x00007FF7E443670A]
	(No symbol) [0x00007FF7E4437964]
	GetHandleVerifier [0x00007FF7E4950AAB+3694587]
	GetHandleVerifier [0x00007FF7E49A728E+4048862]
	GetHandleVerifier [0x00007FF7E499F173+4015811]
	GetHandleVerifier [0x00007FF7E46747D6+695590]
	(No symbol) [0x00007FF7E4550CE8]
	(No symbol) [0x00007FF7E454CF34]
	(No sym

C:\Users\Aidan\AppData\Local\Temp\ipykernel_14884\3273120891.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_14884\3273120891.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_14884\3273120891.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_14884\3273120891.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = result

C:\Users\Aidan\AppData\Local\Temp\ipykernel_14884\3273120891.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_14884\3273120891.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_14884\3273120891.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_14884\3273120891.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = result

C:\Users\Aidan\AppData\Local\Temp\ipykernel_14884\3273120891.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_14884\3273120891.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_14884\3273120891.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_14884\3273120891.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = result

...

Error fetching data for 0006.KL: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.105)
Stacktrace:
	GetHandleVerifier [0x00007FF7E45D82B2+55298]
	(No symbol) [0x00007FF7E4545E02]
	(No symbol) [0x00007FF7E44005AB]
	(No symbol) [0x00007FF7E43E0038]
	(No symbol) [0x00007FF7E4466BC7]
	(No symbol) [0x00007FF7E447A15F]
	(No symbol) [0x00007FF7E4461E83]
	(No symbol) [0x00007FF7E443670A]
	(No symbol) [0x00007FF7E4437964]
	GetHandleVerifier [0x00007FF7E4950AAB+3694587]
	GetHandleVerifier [0x00007FF7E49A728E+4048862]
	GetHandleVerifier [0x00007FF7E499F173+4015811]
	GetHandleVerifier [0x00007FF7E46747D6+695590]
	(No symbol) [0x00007FF7E4550CE8]
	(No symbol) [0x00007FF7E454CF34]
	(No symbol) [0x00007FF7E454D062]
	(No symbol) [0x00007FF7E453D3A3]
	BaseThreadInitThunk [0x00007FFD60777344+20]
	RtlUserThreadStart [0x00007FFD611226B1+33]

Error fetching data for 1902.KL: Message: no such window: target window already closed


Error fetching data for 7084.KL: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.105)
Stacktrace:
	GetHandleVerifier [0x00007FF7E45D82B2+55298]
	(No symbol) [0x00007FF7E4545E02]
	(No symbol) [0x00007FF7E44005AB]
	(No symbol) [0x00007FF7E43E0038]
	(No symbol) [0x00007FF7E4466BC7]
	(No symbol) [0x00007FF7E447A15F]
	(No symbol) [0x00007FF7E4461E83]
	(No symbol) [0x00007FF7E443670A]
	(No symbol) [0x00007FF7E4437964]
	GetHandleVerifier [0x00007FF7E4950AAB+3694587]
	GetHandleVerifier [0x00007FF7E49A728E+4048862]
	GetHandleVerifier [0x00007FF7E499F173+4015811]
	GetHandleVerifier [0x00007FF7E46747D6+695590]
	(No symbol) [0x00007FF7E4550CE8]
	(No symbol) [0x00007FF7E454CF34]
	(No symbol) [0x00007FF7E454D062]
	(No symbol) [0x00007FF7E453D3A3]
	BaseThreadInitThunk [0x00007FFD60777344+20]
	RtlUserThreadStart [0x00007FFD611226B1+33]

Error fetching data for 7544.KL: Message: no such window: target window already closed


Error fetching data for 7412.KL: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.105)
Stacktrace:
	GetHandleVerifier [0x00007FF7E45D82B2+55298]
	(No symbol) [0x00007FF7E4545E02]
	(No symbol) [0x00007FF7E44005AB]
	(No symbol) [0x00007FF7E43E0038]
	(No symbol) [0x00007FF7E4466BC7]
	(No symbol) [0x00007FF7E447A15F]
	(No symbol) [0x00007FF7E4461E83]
	(No symbol) [0x00007FF7E443670A]
	(No symbol) [0x00007FF7E4437964]
	GetHandleVerifier [0x00007FF7E4950AAB+3694587]
	GetHandleVerifier [0x00007FF7E49A728E+4048862]
	GetHandleVerifier [0x00007FF7E499F173+4015811]
	GetHandleVerifier [0x00007FF7E46747D6+695590]
	(No symbol) [0x00007FF7E4550CE8]
	(No symbol) [0x00007FF7E454CF34]
	(No symbol) [0x00007FF7E454D062]
	(No symbol) [0x00007FF7E453D3A3]
	BaseThreadInitThunk [0x00007FFD60777344+20]
	RtlUserThreadStart [0x00007FFD611226B1+33]

Error fetching data for 5173.KL: Message: no such window: target window already closed


Error fetching data for 0199.KL: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.105)
Stacktrace:
	GetHandleVerifier [0x00007FF7E45D82B2+55298]
	(No symbol) [0x00007FF7E4545E02]
	(No symbol) [0x00007FF7E44005AB]
	(No symbol) [0x00007FF7E43E0038]
	(No symbol) [0x00007FF7E4466BC7]
	(No symbol) [0x00007FF7E447A15F]
	(No symbol) [0x00007FF7E4461E83]
	(No symbol) [0x00007FF7E443670A]
	(No symbol) [0x00007FF7E4437964]
	GetHandleVerifier [0x00007FF7E4950AAB+3694587]
	GetHandleVerifier [0x00007FF7E49A728E+4048862]
	GetHandleVerifier [0x00007FF7E499F173+4015811]
	GetHandleVerifier [0x00007FF7E46747D6+695590]
	(No symbol) [0x00007FF7E4550CE8]
	(No symbol) [0x00007FF7E454CF34]
	(No symbol) [0x00007FF7E454D062]
	(No symbol) [0x00007FF7E453D3A3]
	BaseThreadInitThunk [0x00007FFD60777344+20]
	RtlUserThreadStart [0x00007FFD611226B1+33]

Error fetching data for 0118.KL: Message: no such window: target window already closed


           Date    Revenue  Capital Expenditures  Gross Dividend  Net Income  \
0    2023-06-30    1066.45                -65.36          -59.94       25.36   
1    2023-03-31     976.92                -33.38            0.00       15.68   
2    2022-12-31     992.42               -111.97          -29.28        3.07   
3    2022-09-30     988.21                -59.93          -29.28       13.66   
4    2022-06-30     943.67                -37.83          -29.28       25.77   
5    2022-03-31     839.98                -17.01            0.00       24.38   
6    2021-12-31     794.48                -50.97          -18.02       30.12   
7    2021-09-30     680.22                -29.56          -18.02        1.44   
8    2023-06-30     156.28                 -2.87           -6.15       10.52   
9    2023-03-31     146.13                 -0.92            0.00        7.11   
10   2022-12-31     127.77                -56.47          -13.84        8.32   
11   2022-09-30     139.14              

In [5]:
# Rename the columns in the DataFrame
results_df = results_df.rename(columns={
    'Revenue': 'revenue',
    'Capital Expenditures': 'capitalExpenditures',
    'Gross Dividend': 'grossDividend',
    'Net Income': 'netIncome',
    'Operating Cash Flow': 'operatingCashFlow',
    'Operating Income': 'operatingIncome',
    'Preferred Dividends': 'preferredDividends',
    'Shares Outstanding': 'sharesOutstanding',
    'Total Equity': 'totalEquity',
    'stock_code': 'stock_code'
})
results_df['capitalExpenditures'] = results_df['Capital Expenditures'].abs()


In [4]:
# Specify the file path where you want to save the CSV file
csv_file_path = 'exports/stock_financial_quarterly.csv'

# Use the to_csv() method to export the DataFrame to a CSV file
results_df.to_csv(csv_file_path, index=False)  # Set index=False to exclude the DataFrame index from the CSV

print(f"DataFrame saved to {csv_file_path}")


DataFrame saved to exports/stock_financial_quarterly.csv


In [8]:
results_df = pd.read_csv('exports/stock_financial_quarterly.csv')
results_df['stock_code'] = results_df['stock_code'].astype(str)
results_df['Date'] = pd.to_datetime(results_df['Date'])
results_df['capitalExpenditures'] = results_df['capitalExpenditures'].abs()

In [13]:
print(results_df.dtypes)

Date                   datetime64[ns]
revenue                       float64
capitalExpenditures           float64
grossDividend                 float64
netIncome                     float64
operatingCashFlow             float64
operatingIncome               float64
preferredDividends            float64
sharesOutstanding             float64
totalEquity                   float64
stock_code                     object
dtype: object


In [13]:
resulting_df = resulting_df.round(3)

# Specify the file path where you want to save the CSV file
csv_file_path = 'exports/stock_ratio_ttm_quarterly.csv'

# Use the to_csv() method to export the DataFrame to a CSV file
resulting_df.to_csv(csv_file_path, index=False)  # Set index=False to exclude the DataFrame index from the CSV

print(f"DataFrame saved to {csv_file_path}")

DataFrame saved to exports/stock_ratio_ttm_quarterly.csv


In [33]:
import pandas as pd

# Sample DataFrame 'calculated_df' with stock_code and Date columns
calculated_df = pd.DataFrame({
    'stock_code': ['ABC', 'DEF', 'GHI'],
    'Date': ['2023-01-15', '2023-02-20', '2023-03-10']
})

# Sample historical prices DataFrame 'historical_prices_df'
historical_prices_df = pd.DataFrame({
    'stock_code': ['ABC', 'DEF', 'GHI', 'ABC', 'DEF', 'GHI'],
    'Date': ['2023-01-05', '2023-02-10', '2023-03-05', '2023-01-25', '2023-02-15', '2023-03-15'],
    'Close': [100, 150, 200, 110, 160, 220]
})

# Convert 'Date' columns to datetime objects
calculated_df['Date'] = pd.to_datetime(calculated_df['Date'])
historical_prices_df['Date'] = pd.to_datetime(historical_prices_df['Date'])

for index, row in calculated_df.iterrows():
    stock_code = row['stock_code']
    date = row['Date']
    
    # Select historical prices for the stock_code and last recorded date in the month
    selected_prices = historical_prices_df[
        (historical_prices_df['stock_code'] == stock_code) &
        (historical_prices_df['Date'].dt.month == date.month)
    ].sort_values(by='Date')
    
    if not selected_prices.empty:
        last_recorded_date = selected_prices.iloc[-1]['Date']
        closing_price = selected_prices[selected_prices['Date'] == last_recorded_date]['Close'].values[0]
        
        print(f'Stock Code: {stock_code}, Date: {last_recorded_date}, Closing Price: {closing_price}')
    else:
        print(f'Stock Code: {stock_code}, No historical prices found for the given month.')

Stock Code: ABC, Date: 2023-01-25 00:00:00, Closing Price: 110
Stock Code: DEF, Date: 2023-02-15 00:00:00, Closing Price: 160
Stock Code: GHI, Date: 2023-03-15 00:00:00, Closing Price: 220
